In [1]:
import cv2
import json

import matplotlib
matplotlib.use('Agg')
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.path import Path

import numpy as np
import os
import os.path as ops
from collections import Iterable

from multiprocessing import Pool
from multiprocessing import Process

# Path

In [2]:
# image_dir = '../bdd100k/images/100k/train/'
# label_paths = '../bdd100k/labels/bdd100k_labels_images_train_highway_lane.json'

# save_image_path = '../bdd100k/images/100k_train/gt_image/'
# save_binary_path = '../bdd100k/images/100k_train/gt_binary_image/'
# save_combine_path = '../bdd100k/images/100k_train/gt_combine_image/'

In [3]:
image_dir = '../bdd100k/images/100k/val/'
label_paths = '../bdd100k/labels/bdd100k_labels_images_val_highway_lane.json'

save_image_path = '../bdd100k/images/100k_val/gt_image/'
save_binary_path = '../bdd100k/images/100k_val/gt_binary_image/'
save_combine_path = '../bdd100k/images/100k_val/gt_combine_image/'

In [4]:
if not ops.exists(save_image_path):
    os.makedirs(save_image_path)
if not ops.exists(save_binary_path):
    os.makedirs(save_binary_path)
if not ops.exists(save_combine_path):
    os.makedirs(save_combine_path)

# Generate binary image

In [5]:
def read_labels(label_path):
    labels = json.load(open(label_path, 'r'))
    if not isinstance(labels, Iterable):
        labels = [labels]
    return labels

In [6]:
label = read_labels(label_paths)

In [7]:
def get_lanes(objects):
    return [o for o in objects
            if 'poly2d' in o and o['category'][:4] == 'lane']

In [8]:
def poly2patch(vertices, types, closed=False, alpha=1., color=None):
    moves = {'L': Path.LINETO,
             'C': Path.CURVE4}
    points = [v for v in vertices]
    codes = [moves[t] for t in types]
    codes[0] = Path.MOVETO

    if closed:
        points.append(points[0])
        codes.append(Path.CLOSEPOLY)

    if color is None:
        color = random_color()

    return mpatches.PathPatch(
        Path(points, codes),
        facecolor=color if closed else 'none',
        edgecolor=color,  # if not closed else 'none',
        lw=1 if closed else 5 * 1, alpha=alpha,
        antialiased=False, snap=True)

In [9]:
def draw_lane_line(frame, ax, binaly=False):
    objects = frame['labels']

    objects = get_lanes(objects)
    if binaly:
        color = np.array([255, 255, 255, 255]) / 255
    else:
        color = np.array([255, 0, 0, 255]) / 255
    alpha = 1.0

    for obj in objects:
        if obj['attributes']['laneDirection'] == 'parallel' and \
           obj['attributes']['laneType'] != 'road curb':
            for poly in obj['poly2d']:
                ax.add_patch(poly2patch(poly['vertices'], poly['types'], closed=poly['closed'],
                                        alpha=alpha, color=color))

In [10]:
def function(i):    
    frame = label[i]
    image_path = image_dir + frame['name']
    img = mpimg.imread(image_path)
    save_name = '{:04d}.png'.format(i+start_number)

    # matplotlib setting
    dpi = 80
    w = 16
    h = 9
    fig = plt.figure(figsize=(w, h), dpi=dpi)
    ax = fig.add_axes([0.0, 0.0, 1.0, 1.0], frameon=False, polar=False, projection=None)

    # img
    img_uint8 = np.array(img, dtype=np.uint8)
    ax.imshow(img_uint8, interpolation='nearest', aspect='auto')
    ax.axis('off')
    if save:
        plt.savefig(save_image_path + save_name)

    # binary lane img
    img_uint8_zero = np.zeros([img.shape[0], img.shape[1], img.shape[2]], np.uint8)
    ax.imshow(img_uint8_zero, interpolation='nearest', aspect='auto', cmap=plt.cm.gray)
    draw_lane_line(frame, ax, binaly=True)
    ax.axis('off')
    if save:
        plt.savefig(save_binary_path + save_name)

    # img + lane
    img_uint8 = np.array(img, dtype=np.uint8)
    ax.imshow(img_uint8, interpolation='nearest', aspect='auto')
    draw_lane_line(frame, ax)
    ax.axis('off')
    if save:
        plt.savefig(save_combine_path + save_name)

    plt.clf()
    plt.close('all')

    return

In [11]:
start_number = 0
save = True
max_processes = 8

n = len(label)
# n = 10

In [12]:
p = Pool(max_processes)
p.map(function, range(n))
p.close()
p.terminate()